In [6]:
import numpy as np

# ------------------------------
# Neural Network Model
# ------------------------------
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size):
        # Number of weights
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_weights = (input_size * hidden_size) + hidden_size + (hidden_size * output_size) + output_size

    def forward(self, x, weights):
        # unpack weights
        ih_end = self.input_size * self.hidden_size
        ih = weights[:ih_end].reshape(self.input_size, self.hidden_size)
        hb = weights[ih_end:ih_end + self.hidden_size]

        ho_start = ih_end + self.hidden_size
        ho_end = ho_start + self.hidden_size * self.output_size
        ho = weights[ho_start:ho_end].reshape(self.hidden_size, self.output_size)
        ob = weights[ho_end:]

        # forward pass
        hidden = np.tanh(np.dot(x, ih) + hb)
        output = 1 / (1 + np.exp(-(np.dot(hidden, ho) + ob)))  # sigmoid
        return output

    def fitness(self, X, y, weights):
        predictions = self.forward(X, weights)
        return np.mean((predictions - y) ** 2)  # MSE


# ------------------------------
# Levy flight function
# ------------------------------
def levy_flight(Lambda=1.5):
    u = np.random.normal(0, 1)
    v = np.random.normal(0, 1)
    step = u / (abs(v) ** (1 / Lambda))
    return step


# ------------------------------
# Cuckoo Search for NN Training
# ------------------------------
def cuckoo_search(nn, X, y, n=30, max_iter=1000, pa=0.25, step_size=0.1):
    # Initialize nests
    nests = np.random.randn(n, nn.num_weights)
    fitness = np.array([nn.fitness(X, y, nest) for nest in nests])

    for t in range(max_iter):
        for i in range(n):
            # Generate new solution by Levy flight
            new_nest = nests[i] + step_size * levy_flight() * np.random.randn(nn.num_weights)
            new_nest = np.clip(new_nest, -5, 5)  # <-- prevents huge weights
            new_fitness = nn.fitness(X, y, new_nest)

            # If better, replace
            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

        # Abandon some nests with probability pa
        abandon = np.random.rand(n) < pa
        nests[abandon] = np.random.randn(np.sum(abandon), nn.num_weights)
        fitness[abandon] = [nn.fitness(X, y, nest) for nest in nests[abandon]]

        # Track best solution
        best_idx = np.argmin(fitness)
        if (t+1) % 50 == 0:  # Print every 50 iterations
            print(f"Iteration {t+1}/{max_iter}, Best fitness: {fitness[best_idx]:.4f}")

    return nests[best_idx], fitness[best_idx]


# ------------------------------
# XOR Problem
# ------------------------------
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])

# Create NN (larger hidden size helps with XOR)
nn = SimpleNN(input_size=2, hidden_size=10, output_size=1)

# Train using Cuckoo Search
best_weights, best_error = cuckoo_search(nn, X, y, n=30, max_iter=1000, step_size=0.1)

print("\nFinal Best Error:", best_error)

# ------------------------------
# Test Predictions
# ------------------------------
predictions = nn.forward(X, best_weights)
for inp, pred in zip(X, predictions):
    print(f"{inp} -> {pred[0]:.4f}")


Iteration 50/1000, Best fitness: 0.2508
Iteration 100/1000, Best fitness: 0.1513
Iteration 150/1000, Best fitness: 0.2112
Iteration 200/1000, Best fitness: 0.2209
Iteration 250/1000, Best fitness: 0.2587
Iteration 300/1000, Best fitness: 0.2322
Iteration 350/1000, Best fitness: 0.2205
Iteration 400/1000, Best fitness: 0.2289
Iteration 450/1000, Best fitness: 0.1697
Iteration 500/1000, Best fitness: 0.2302
Iteration 550/1000, Best fitness: 0.2546
Iteration 600/1000, Best fitness: 0.2091
Iteration 650/1000, Best fitness: 0.2177
Iteration 700/1000, Best fitness: 0.1527
Iteration 750/1000, Best fitness: 0.2083
Iteration 800/1000, Best fitness: 0.2265
Iteration 850/1000, Best fitness: 0.2490
Iteration 900/1000, Best fitness: 0.1987
Iteration 950/1000, Best fitness: 0.1866
Iteration 1000/1000, Best fitness: 0.1494

Final Best Error: 0.14939775334350514
[0 0] -> 0.4786
[0 1] -> 0.5835
[1 0] -> 0.6605
[1 1] -> 0.2825
